In [1]:
import os
import os.path as osp
import sys
sys.path.insert(0, osp.dirname(osp.abspath('.')))

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from data.dataset import COCO_CLASSES

from PIL import Image 
from pycocotools.coco import COCO
from pycocotools.mask import encode, decode, area, toBbox

## Sanity Checks

Make sure data downloaded from the Kaggle is the same as data provided on official COCO website.

In [2]:
def count_kaggle_images():
    files = os.listdir("../download/COCO/images/")
    train_images = [ f for f in files if 'train' in f ]
    valid_images = [ f for f in files if 'val' in f ]
    return len(train_images), len(valid_images)

def count_kaggle_annots():
    files = os.listdir("../download/COCO/labels/")
    train_labels = [ f for f in files if 'train' in f ]
    valid_labels = [ f for f in files if 'val' in f ]
    return len(train_labels), len(valid_labels)

def count_train_split():
    imgs = []
    labels = []
    with open("../download/COCO/train.csv", 'r') as f:
        lines = f.readlines()
        for line in lines:
            img, label = line.split(',')
            imgs.append(img)
            labels.append(label)
    return len(lines)

def count_test_split():
    imgs = []
    labels = []
    with open("../download/COCO/test.csv", 'r') as f:
        lines = f.readlines()
        for line in lines:
            img, label = line.split(',')
            imgs.append(img)
            labels.append(label)
    return len(lines)


print("IMAGES:", count_kaggle_images())
print("ANNOTS:", count_kaggle_annots())
print("===============================")
print("TRAIN:", count_train_split())
print("TEST:", count_test_split())

IMAGES: (82783, 40504)
ANNOTS: (82081, 40137)
TRAIN: 117264
TEST: 4954


## Export Instance Segmentation Masks

In [3]:
Ids = list(range(10))

n_tasks = 3
n_loads = len(Ids)//n_tasks

for i in range(n_tasks):
    if i == n_tasks - 1:
        print(Ids[i*n_loads:])
    else:
        print(Ids[i*n_loads:i*n_loads+n_loads])

[0, 1, 2]
[3, 4, 5]
[6, 7, 8, 9]


In [4]:
a = 1.134 


In [5]:
round(a, ndigits=2)

1.13

In [6]:
from multiprocessing import Process

def export_worker(coco, output_dir, imgIds):
    for image_id in imgIds:
        # Get Image name
        image_metadata = coco.loadImgs(ids=[image_id])[0]
        image_file = image_metadata['file_name']
        
        # Create Subdirectory
        subdir = osp.join(output_dir, image_file)
        if not osp.exists(subdir):
            os.makedirs(subdir)
            
        annIds = coco.getAnnIds(imgIds=[image_id])
        annots = coco.loadAnns(annIds)
        mask_count = 0
        for annot in annots:
            if annot['iscrowd']:
                continue
            # Construct binary mask
            mask = coco.annToMask(annot)
            img = Image.fromarray(np.uint8(mask*255), 'L')
            # Construct bounding box
            img_width, img_height = image_metadata['width'], image_metadata['height']
            bbox = annot['bbox']
            cx = max([bbox[0]+(bbox[2]/2)-1, 0.0])
            cy = max([bbox[1]+(bbox[3]/2)-1, 0.0])
            normalized = [ 
                cx/img_width,
                cy/img_height,
                min([bbox[2]/img_width, 1.0]),
                min([bbox[3]/img_height, 1.0]),
                ]
            cat = coco.loadCats(ids=[annot['category_id']])[0]
            cls = COCO_CLASSES.index(cat['name'])
            normalized.insert(0, cls)
            
            if (
                (abs(
                    round(normalized[1]-(normalized[3]/2), ndigits=5)
                    - round(normalized[1]+(normalized[3]/2), ndigits=5)
                ) < 0.01) 
                or 
                (abs(
                    round(normalized[2]-(normalized[4]/2), ndigits=5)
                    - round(normalized[2]+(normalized[4]/2), ndigits=5)
                ) < 0.01)
            ):
                continue

            # Export mask
            prefix, suffix = image_file.split(".") # E.g. COCO_val2014_000000184613.jpg
            mask_name = f"{prefix}_mask_{mask_count}.{suffix}"
            img.save(osp.join(subdir, mask_name))
            # Export bbox
            bbox_name = f"{prefix}_mask_{mask_count}.txt"
            with open(osp.join(subdir, bbox_name), 'w') as f:
                fields = [ str(v) for v in normalized ]
                fields[0] = str(int(normalized[0]))
                f.write(" ".join(fields))
            mask_count += 1
            
def export_instance_masks(annFile, output_dir):
    # Create directory Structure
    if not osp.exists(output_dir):
        os.makedirs(output_dir)
        
    # Read annotation file
    coco = COCO(annFile)
    
    # Process all images
    imgIds = coco.getImgIds()

    procs = []
    n_tasks = 64
    n_loads = len(imgIds)//n_tasks
    for i in range(n_tasks):
        if i == (n_tasks-1):
            subIds = imgIds[i*n_loads:]
        else:
            subIds = imgIds[i*n_loads:i*n_loads+n_loads]
        proc = Process(target=export_worker, args=(coco, output_dir, subIds))
        proc.start()
        procs.append(proc)
    
    for proc in procs:
        proc.join()

In [7]:
export_instance_masks("annotations/instances_val2014.json", "../download/COCO/masks")

loading annotations into memory...
Done (t=4.89s)
creating index...
index created!


In [8]:
export_instance_masks("annotations/instances_train2014.json", "../download/COCO/masks")

loading annotations into memory...
Done (t=10.53s)
creating index...
index created!
